In [ ]:
from urllib.request import urlopen
import ssl
import json

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

dataTrain = urlopen("https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/train/dialogues_001.json", context=ctx).read().decode()
dataTest = urlopen("https://raw.githubusercontent.com/budzianowski/multiwoz/master/data/MultiWOZ_2.2/test/dialogues_001.json", context=ctx).read().decode()

myJsonTrain = json.loads(dataTrain)
myJsonTest = json.loads(dataTest)
print (json.dumps(myJsonTrain[0], indent=1)[0:100])
print (json.dumps(myJsonTest[0], indent=1)[0:100])



{
 "dialogue_id": "PMUL4398.json",
 "services": [
  "restaurant",
  "hotel"
 ],
 "turns": [
  {
   "
{
 "dialogue_id": "MUL0484.json",
 "services": [
  "attraction",
  "train"
 ],
 "turns": [
  {
   "f


In [ ]:
#http://alexminnaar.com/2019/08/22/ner-rnns-tensorflow.html
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

import numpy as np
import os
import time

print(tf.__version__)

2.5.0


In [ ]:
import re

from nltk.util import ngrams

timepat = re.compile("\d{1,2}[:]\d{1,2}")
pricepat = re.compile("\d{1,3}[.]\d{1,2}")

fin = urlopen("https://raw.githubusercontent.com/budzianowski/multiwoz/master/utils/mapping.pair", context=ctx)
replacements = []
for line in fin.readlines():
    line = line.decode().replace('\n', '')
    tok_from, tok_to = line.replace('\n', '').split('\t')
    replacements.append((' ' + tok_from + ' ', ' ' + tok_to + ' '))


def insertSpace(token, text):
    sidx = 0
    while True:
        sidx = text.find(token, sidx)
        if sidx == -1:
            break
        if sidx + 1 < len(text) and re.match('[0-9]', text[sidx - 1]) and \
                re.match('[0-9]', text[sidx + 1]):
            sidx += 1
            continue
        if text[sidx - 1] != ' ':
            text = text[:sidx] + ' ' + text[sidx:]
            sidx += 1
        if sidx + len(token) < len(text) and text[sidx + len(token)] != ' ':
            text = text[:sidx + 1] + ' ' + text[sidx + 1:]
        sidx += 1
    return text


def normalize(text):
    # lower case every word
    text = text.lower()

    # replace white spaces in front and end
    text = re.sub(r'^\s*|\s*$', '', text)

    # hotel domain pfb30
    text = re.sub(r"b&b", "bed and breakfast", text)
    text = re.sub(r"b and b", "bed and breakfast", text)

    # normalize phone number
    ms = re.findall('\(?(\d{3})\)?[-.\s]?(\d{3})[-.\s]?(\d{4,5})', text)
    if ms:
        sidx = 0
        for m in ms:
            sidx = text.find(m[0], sidx)
            if text[sidx - 1] == '(':
                sidx -= 1
            eidx = text.find(m[-1], sidx) + len(m[-1])
            text = text.replace(text[sidx:eidx], ''.join(m))

    # normalize postcode
    ms = re.findall('([a-z]{1}[\. ]?[a-z]{1}[\. ]?\d{1,2}[, ]+\d{1}[\. ]?[a-z]{1}[\. ]?[a-z]{1}|[a-z]{2}\d{2}[a-z]{2})',
                    text)
    if ms:
        sidx = 0
        for m in ms:
            sidx = text.find(m, sidx)
            eidx = sidx + len(m)
            text = text[:sidx] + re.sub('[,\. ]', '', m) + text[eidx:]

    # weird unicode bug
    text = re.sub(u"(\u2018|\u2019)", "'", text)

    # replace time and and price
    times = re.findall(timepat, text)
    prices = re.findall(pricepat, text)
    text = re.sub(timepat, ' [value_time] ', text)
    text = re.sub(pricepat, ' [value_price] ', text)
    #text = re.sub(pricepat2, '[value_price]', text)

    # replace st.
    text = text.replace(';', ',')
    text = re.sub('$\/', '', text)
    text = text.replace('/', ' and ')

    # replace other special characters
    text = text.replace('-', ' ')
    text = re.sub('[\":\<>@\(\)]', '', text)

    # insert white space before and after tokens:
    for token in ['?', '.', ',', '!']:
        text = insertSpace(token, text)

    # insert white space for 's
    text = insertSpace('\'s', text)

    # replace it's, does't, you'd ... etc
    text = re.sub('^\'', '', text)
    text = re.sub('\'$', '', text)
    text = re.sub('\'\s', ' ', text)
    text = re.sub('\s\'', ' ', text)
    for fromx, tox in replacements:
        text = ' ' + text + ' '
        text = text.replace(fromx, tox)[1:-1]

    # remove multiple spaces
    text = re.sub(' +', ' ', text)

    # concatenate numbers
    tmp = text
    tokens = text.split()
    i = 1
    while i < len(tokens):
        if re.match(u'^\d+$', tokens[i]) and \
                re.match(u'\d+$', tokens[i - 1]):
            tokens[i - 1] += tokens[i]
            del tokens[i]
        else:
            i += 1
    text = ' '.join(tokens)

    for time in times:
      text = re.sub('\\[value_time\\]', time, text, 1)

    for price in prices:
      text = re.sub('\\[value_price\\]', price, text, 1)

    return text

In [72]:
# filter
import string

def filter(data):
  result = []

  for i, obj in enumerate(data):
      turns = obj.get("turns", None)
      for turn in turns:
          if turn:
              utterance = turn.get("utterance", None)
              if utterance:
                  frames = turn.get("frames", None)
                  if frames:
                      dic = dict()
                      res = [[], []]
                      for frame in frames: 
                          state = frame.get("state", None)
                          service = frame.get("service", None)
                          if service and service == "restaurant" and state:
                              slot_values = state.get("slot_values", None)
                              if slot_values and len(slot_values) != 0:
                                  for key, value in slot_values.items():
                                      for elem in value:
                                        dic[elem] = key


                      text = normalize(utterance)
                      for word in text.split(" "):
                          res[0].append(word)
                          res[1].append(dic.get(word, "other"))

                      result.append(res)




  return result

train_dataset = filter(myJsonTrain)
for i, elem in enumerate(train_dataset):
      # print(elem)
      if i > 30:
        # print("--------------------")
        break
test_dataset = filter(myJsonTest)
for i, elem in enumerate(test_dataset):
      # print(elem)
      if i > 30:
        # print("--------------------")
        break


test_dataset[6:9]

[[['can',
   'you',
   'book',
   'me',
   'a',
   'table',
   'for',
   '11:00',
   'on',
   'friday',
   '?'],
  ['other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'restaurant-booktime',
   'other',
   'restaurant-bookday',
   'other']],
 [['actually', ',', 'for', '4', ',', 'please', '.'],
  ['other',
   'other',
   'other',
   'restaurant-bookpeople',
   'other',
   'other',
   'other']],
 [['great',
   ',',
   'can',
   'you',
   'also',
   'get',
   'me',
   'information',
   'or',
   'architecture',
   'in',
   'the',
   'area'],
  ['other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'other',
   'other']]]